In [ ]:
from yaw import UniformRandoms

from rail.yaw_rail import (
    YawAutoCorrelate,
    YawCacheCreate,
    YawCacheDrop,
    YawCrossCorrelate,
    YawSummarize,
)  # equivalent: from rail.yaw_rail import *
from rail.yaw_rail.utils import get_dc2_test_data

In [ ]:
VERBOSE = "debug"

In [ ]:
mock_data = get_dc2_test_data()  # downloads test data, cached for future calls
redshifts = mock_data["z"].to_numpy()
zmin = redshifts.min()
zmax = redshifts.max()
n_data = len(mock_data)

In [ ]:
angular_rng = UniformRandoms(
    mock_data["ra"].min(),
    mock_data["ra"].max(),
    mock_data["dec"].min(),
    mock_data["dec"].max(),
    seed=12345,
)
mock_rand = angular_rng.generate(n_data * 10, draw_from=dict(z=redshifts))

In [ ]:
cache_ref = YawCacheCreate.make_stage(
    name="ref",
    path="./test_ref",
    ra_name="ra",
    dec_name="dec",
    redshift_name="z",
    # weight_name=,
    # patches=,
    # patch_name=,
    n_patches=5,
    verbose=VERBOSE,  # default: "info"
).create(
    data=mock_data,
    rand=mock_rand,
)

In [ ]:
cache_unk = YawCacheCreate.make_stage(
    name="unk",
    path="./test_unk",
    ra_name="ra",
    dec_name="dec",
    # weight_name=,
    patches="./test_ref",
    # patch_name=,
    # n_patches=,
    verbose=VERBOSE,  # default: "info"
).create(
    data=mock_data,
    rand=mock_rand,
)

In [ ]:
corr_config = dict(
    rmin=100,   # in kpc
    rmax=1000,  # in kpc
    # rweight=None,
    # rbin_num=50,
    zmin=zmin,
    zmax=zmax,
    zbin_num=8,  # default: 30
    # method="linear",
    # zbins=np.linspace(zmin, zmax, zbin_num+1)
    # crosspatch=True,
    # thread_num=None,
    verbose=VERBOSE,  # default: "info"
)

In [ ]:
w_ss = YawAutoCorrelate.make_stage(
    name="w_ss",
    **corr_config
).correlate(
    sample=cache_ref,
)

In [ ]:
w_sp = YawCrossCorrelate.make_stage(
    name="w_sp",
    **corr_config
).correlate(
    reference=cache_ref,
    unknown=cache_unk,
)

In [ ]:
estimate = YawSummarize.make_stage(
    # cross_est=,
    # ref_est=,
    # unk_est=,
    # crosspatch=True,
    verbose=VERBOSE,  # default: "info"
).summarize(
    cross_corr=w_sp,
    ref_corr=w_ss,
)

ax = estimate["output"].data.plot(xlim=[zmin, zmax])
mock_data.hist("z", bins=w_sp.data.edges, density=True, color="0.8", ax=ax)
estimate["yaw_cc"].data.normalised().plot()

In [ ]:
YawCacheDrop.make_stage().drop(cache_unk)
YawCacheDrop.make_stage().drop(cache_ref)